# Data analysis

**Goal**: Analyse the differences in hotel costs and per diems of employees of the commission

1. Import and explore the data we have scraped in the previous excercise.
2. Clean this data
3. Combine them with the [inflation index from Eurostat](https://ec.europa.eu/eurostat/databrowser/bookmark/f6a583fa-f744-4590-aa95-173aaa6ea3f1?lang=en) (grab the [direct link to the csv](https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/prc_hicp_midx/1.0/M.I05.CP11.*?c[geo]=BE,BG,CZ,DK,DE,EE,IE,EL,ES,FR,HR,IT,CY,LV,LT,LU,HU,MT,NL,AT,PL,PT,RO,SI,SK,FI,SE,IS,NO,CH,UK,ME,MK&compress=false&format=csvdata&formatVersion=2.0&c[TIME_PERIOD]=ge:2004-01+le:2023-10&lang=en&labels=name))
4. Analyse the data

##  1. Import and explore the scraped data

The first step for a data analysis is to have a close look at the data:
- What columns are there?
- How many rows?
- What is in the columns?

Pandas has helpful methods to do this.

```python
df.info() # information on the column names and non-zero values
df.sample(3) # returns 3 sample rows from the data
df.head(3) # returns the first 3 rows of the data
len(df) # returns the number of rows
df.columns # all the column names as a list
```

In [145]:
import pandas as pd

# supress the warnings when modifying a column
pd.options.mode.chained_assignment = None

In [108]:
df = pd.read_csv("data/regulation_data.csv")
df.sample(3)

,Destination,Hotel ceiling,Daily allowance,Date
183,Luxembourg,148,98,20180101
35,Greece,112,82,20220701
525,Portugal,120,84,20070501


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Destination      636 non-null    object
 1   Hotel ceiling    636 non-null    object
 2   Daily allowance  636 non-null    object
 3   Date             636 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 20.0+ KB


### `df.info()` explained

![](img/info.png)

### Looking closely at the columns

To isolate columns using this syntax and look more closely at those we use the following syntax:

```python
df["column"]
```

This will return a `Series` from a `DataFrame`. It's important to know what data type you are working with, as all of them have their own methods.


```python
# for categorical variables
df["column"].unique() # all the unique values of the column
df["column"].value_counts() # how often does a value occur

# for numeric variables
df.hist(column='column') # makes a histogram
df.describe() # descriptive statistics for all numeric variables
df["column"].describe() # descriptive statistics for a single column
```

In [110]:
df["Destination"].value_counts()

Destination
Belgium            23
Latvia             23
United Kingdom     23
Sweden             23
Finland            23
Slovenia           23
Portugal           23
Poland             23
Austria            23
Netherlands        23
Malta              23
Hungary            23
Lithuania          23
Luxembourg         23
Cyprus             23
Germany            23
Italy              23
Czech Republic     23
France             23
Spain              23
Greece             23
Ireland            23
Denmark            23
Estonia            23
Romania            19
Bulgaria           19
Slovakia           14
Destination        11
Slovak Republic     9
Croatia             9
Destinations        3
Name: count, dtype: int64

This counts the values in the column. Note that we have the words *Destination* and *Destinations* in there. These are remnants of the headers, we don't want them. We also have *Slovakia* and *Slovak Republic* as a value. We need to make them one.

## 2. Cleaning the data

### Filtering the DataFrame

```python
df.query() # filters the DataFrame
```

- [see more example usages of `.query()`](https://github.com/zufanka/2023-GUN_MIJ/blob/main/resources/query_example_usage.md)

In [146]:
# return only the rows that do not contain "Destinations" or "Destination" in the 'Destination' column
df2 = df.query("~Destination.str.contains('Destinations?')")

* `~` except
* `str.contains()` = pandas [str.contains()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html)
* ('Destinations **?**') = ? means that the previous character can be there but does not have to be. Therefore this matches both "Destinations" and "Destination". This syntax comes from [regular expressions](https://regexr.com/)

We also need to replace the "Slovak Republic" with "Slovakia".

In [147]:
df2.loc[:,"Destination"] = df2["Destination"].str.replace("Slovak Republic", "Slovakia")

# checking if the changes took place
df2["Destination"].value_counts()

Destination
Belgium           23
Latvia            23
Sweden            23
Finland           23
Slovakia          23
Slovenia          23
Portugal          23
Poland            23
Austria           23
Netherlands       23
Malta             23
Hungary           23
Luxembourg        23
Lithuania         23
Cyprus            23
Italy             23
France            23
Spain             23
Greece            23
Ireland           23
Estonia           23
Germany           23
Denmark           23
Czech Republic    23
United Kingdom    23
Bulgaria          19
Romania           19
Croatia            9
Name: count, dtype: int64

Next we need to change the `dtypes` of the columns:
- `Hotel ceiling` and `Daily allowance` to `float` instead of `object`
- `Date` to `date` instead of `int`

In [148]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 622 entries, 0 to 635
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Destination      622 non-null    object
 1   Hotel ceiling    622 non-null    object
 2   Daily allowance  622 non-null    object
 3   Date             622 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 24.3+ KB


Use the following functions to change the `dtype` from `object` to `int64` or `float64`

```python
pd.to_numeric() # change text to numbers
```

In [149]:
# we use .str.replace() here to replace the , with . as , is not a valid decimal separator in python
df2["Hotel ceiling"] = pd.to_numeric(df2["Hotel ceiling"].str.replace(",","."))
df2["Daily allowance"] = pd.to_numeric(df2["Daily allowance"].str.replace(",","."))

Do you see this? It means that you have executed the cell twice and the columns are already numbers and can not be converted into numbers again
![attribute_error](img/attribute_error.png)

In [152]:
# checking if the two columns have the correct dtype
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 622 entries, 0 to 635
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Destination      622 non-null    object 
 1   Hotel ceiling    622 non-null    float64
 2   Daily allowance  622 non-null    float64
 3   Date             622 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 24.3+ KB


In [153]:
# Diving into descriptive statistics
df2.describe()

,Hotel ceiling,Daily allowance,Date
count,622.000000,622.000000,6.220000e+02
mean,138.182781,85.243424,2.013750e+07
std,23.498804,17.272073,5.925305e+04
min,97.030000,50.000000,2.004050e+07
25%,117.000000,72.000000,2.009010e+07
50%,139.660000,86.890000,2.014050e+07
75%,150.000000,97.000000,2.019010e+07
max,210.000000,125.000000,2.023010e+07


Most often we want to group values of a column and calculate a `sum`, `mean` or something else.
For this we use

```python
df.groupby("city").mean() # groups by the column "city" and returns the average values for all numeric columns
```

- [see more example usage of `.groupby()`](https://github.com/zufanka/2023-GUN_MIJ/blob/main/resources/groupby_example_usage.md)

In [154]:
# average value of all the numeric values grouped by the country, sorted from largest to smallest on the 'Hotel ceiling' column
df2.groupby("Destination").mean().sort_values("Hotel ceiling", ascending=False)

,Hotel ceiling,Daily allowance,Date
Destination,,,
United Kingdom,184.916957,108.550870,2.013553e+07
Sweden,168.187391,104.292609,2.013553e+07
Netherlands,163.446957,94.990435,2.013553e+07
Denmark,158.748261,117.873913,2.013553e+07
France,154.861304,94.814783,2.013553e+07
Romania,153.894737,56.736842,2.015293e+07
Bulgaria,152.894737,57.526316,2.015293e+07
Ireland,152.128696,102.557391,2.013553e+07
Czech Republic,145.478261,70.434783,2.013553e+07


The above calculation however does not make sense as the data comes from various years and the euro value in 2004 is different than in 2023 due to inflation. We therefore need to adjust this data for inflation first. For that we will pull the [Inflation index from Eurostat](https://ec.europa.eu/eurostat/databrowser/bookmark/f6a583fa-f744-4590-aa95-173aaa6ea3f1?lang=en), filtered on the 'Restaurants and Hotels' inflation.

We use the following function to change the `dtype` to `datetime64`

```python
pd.to_datetime() # change text or number to a date format
```

We have to specify the format our date is in in order for `pandas` to read it correctly. This date format is called `strftime`. Each symbol followed by a character in strftime represents a different component of the date or time. For example:

- `%Y-%m-%d` will output a date in the format of '2023-12-13' 
- `%B %d, %Y` will output a date in the format of 'December 13, 2023'
- `%H:%M:%S` will output the current time in the format of '15:30:45'

See also the [strftime reference cheatsheet](https://strftime.org/)

In [162]:
df2['Date'] = pd.to_datetime(df2['Date'], format="%Y%m%d")

In [164]:
# check if the format is correct
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 622 entries, 0 to 635
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Destination      622 non-null    object        
 1   Hotel ceiling    622 non-null    float64       
 2   Daily allowance  622 non-null    float64       
 3   Date             622 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 24.3+ KB


## 3. Importing the inflation dataset
You can grab the [direct link to the csv here](https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/prc_hicp_midx/1.0/M.I05.CP11.*?c[geo]=BE,BG,CZ,DK,DE,EE,IE,EL,ES,FR,HR,IT,CY,LV,LT,LU,HU,MT,NL,AT,PL,PT,RO,SI,SK,FI,SE,IS,NO,CH,UK,ME,MK&compress=false&format=csvdata&formatVersion=2.0&c[TIME_PERIOD]=ge:2004-01+le:2023-10&lang=en&labels=name) directly into the pandas function, or save the file in the same folder as your notebook

In [214]:
df_inflation = pd.read_csv("data/inflation_data.csv")

# .T transposes the data so we can read it better
df_inflation.sample(3).T

,635,2481,5222
STRUCTURE,dataflow,dataflow,dataflow
STRUCTURE_ID,ESTAT:PRC_HICP_MIDX(1.0),ESTAT:PRC_HICP_MIDX(1.0),ESTAT:PRC_HICP_MIDX(1.0)
STRUCTURE_NAME,HICP - monthly data (index),HICP - monthly data (index),HICP - monthly data (index)
freq,M,M,M
Time frequency,Monthly,Monthly,Monthly
unit,I05,I05,I05
Unit of measure,"Index, 2005=100","Index, 2005=100","Index, 2005=100"
coicop,CP11,CP11,CP11
Classification of individual consumption by purpose (COICOP),Restaurants and hotels,Restaurants and hotels,Restaurants and hotels
geo,BG,ES,MT


In [215]:
df_inflation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7559 entries, 0 to 7558
Data columns (total 17 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   STRUCTURE                                                     7559 non-null   object 
 1   STRUCTURE_ID                                                  7559 non-null   object 
 2   STRUCTURE_NAME                                                7559 non-null   object 
 3   freq                                                          7559 non-null   object 
 4   Time frequency                                                7559 non-null   object 
 5   unit                                                          7559 non-null   object 
 6   Unit of measure                                               7559 non-null   object 
 7   coicop                                                        7559 no

In [216]:
for column in df_inflation.columns:
    display(df_inflation[column].unique())

array(['dataflow'], dtype=object)

array(['ESTAT:PRC_HICP_MIDX(1.0)'], dtype=object)

array(['HICP - monthly data (index)'], dtype=object)

array(['M'], dtype=object)

array(['Monthly'], dtype=object)

...

array([nan])

array([ 96.99,  97.86,  97.64, ..., 151.8 , 150.8 , 146.7 ])

array([nan])

array([nan, 'u', 'd', 'du'], dtype=object)

array([nan, 'low reliability', 'definition differs (see metadata)',
       'definition differs (see metadata), low reliability'], dtype=object)

In [217]:
df_inflation.columns

Index(['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'freq', 'Time frequency',
       'unit', 'Unit of measure', 'coicop',
       'Classification of individual consumption by purpose (COICOP)', 'geo',
       'Geopolitical entity (reporting)', 'TIME_PERIOD', 'Time', 'OBS_VALUE',
       'Observation value', 'OBS_FLAG', 'Observation status (Flag)'],
      dtype='object')

In [218]:
drop_these = ['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'freq', 'Time frequency',
       'unit', 'Unit of measure', 'coicop','Observation value',
       'Classification of individual consumption by purpose (COICOP)', 'geo','Time',
            'OBS_FLAG']

df_inflation = df_inflation.drop(drop_these, axis=1)

In [219]:
df_inflation.sample(3)

,Geopolitical entity (reporting),TIME_PERIOD,OBS_VALUE,Observation status (Flag)
6194,Portugal,2006-05,102.07,NaN
2033,Estonia,2015-09,156.17,NaN
7017,Slovenia,2015-06,135.45,NaN


In [220]:
df_inflation = df_inflation.rename(columns={
    "Geopolitical entity (reporting)" : "Country",
    "OBS_VALUE" : "inflation_index"
})

In [221]:
df_inflation\
.query('`Observation status (Flag)`.notnull()')\
.groupby("Country")['Observation status (Flag)']\
.count()

Country
Austria             10
Belgium              8
Bulgaria             1
Croatia              3
Cyprus               1
Czechia              1
Denmark              6
France              10
Germany              8
Greece               4
Hungary              4
Ireland              9
Italy                3
Lithuania            2
Luxembourg           5
Netherlands          2
North Macedonia    227
Poland               5
Portugal             2
Romania              5
Slovakia             2
Slovenia             3
Spain                2
Switzerland          5
United Kingdom       5
Name: Observation status (Flag), dtype: int64

In [222]:
df_inflation["Country"].value_counts()

Country
Austria            238
Belgium            238
Slovakia           238
Slovenia           238
Sweden             238
Romania            238
Portugal           238
Poland             238
Norway             238
Netherlands        238
Malta              238
Latvia             238
Luxembourg         238
Lithuania          238
Italy              238
Iceland            238
Ireland            238
Hungary            238
Croatia            238
France             238
Finland            238
Spain              238
Greece             238
Estonia            238
Denmark            238
Germany            238
Czechia            238
Cyprus             238
Bulgaria           238
North Macedonia    227
Switzerland        227
United Kingdom     203
Name: count, dtype: int64

In [223]:
df2["date_short"] = df2["Date"].dt.strftime("%Y-%m")

In [228]:
df_adjusted = pd.merge(
    df2,
    df_inflation,
    left_on = ["Destination", "date_short"],
    right_on = ["Country", "TIME_PERIOD"],
    how = "left"
)

In [225]:
df_adjusted.sample(3)

,Destination,Hotel ceiling,Daily allowance,Date,date_short,Country,TIME_PERIOD,inflation_index,Observation status (Flag)
118,Ireland,159.0,108.0,2020-01-01,2020-01,Ireland,2020-01,124.80,NaN
335,Czech Republic,155.0,75.0,2014-01-01,2014-01,NaN,NaN,NaN,NaN
533,Latvia,145.0,66.0,2006-07-01,2006-07,Latvia,2006-07,111.87,NaN


In [226]:
df_adjusted.query("inflation_index.isnull()")

,Destination,Hotel ceiling,Daily allowance,Date,date_short,Country,TIME_PERIOD,inflation_index,Observation status (Flag)
2,Czech Republic,124.0,70.0,2023-01-01,2023-01,NaN,NaN,NaN,NaN
27,United Kingdom,209.0,125.0,2023-01-01,2023-01,NaN,NaN,NaN,NaN
30,Czech Republic,124.0,70.0,2022-07-01,2022-07,NaN,NaN,NaN,NaN
55,United Kingdom,209.0,125.0,2022-07-01,2022-07,NaN,NaN,NaN,NaN
58,Czech Republic,124.0,70.0,2022-01-01,2022-01,NaN,NaN,NaN,NaN
83,United Kingdom,209.0,125.0,2022-01-01,2022-01,NaN,NaN,NaN,NaN
86,Czech Republic,124.0,70.0,2021-01-01,2021-01,NaN,NaN,NaN,NaN
111,United Kingdom,209.0,125.0,2021-01-01,2021-01,NaN,NaN,NaN,NaN
114,Czech Republic,124.0,70.0,2020-01-01,2020-01,NaN,NaN,NaN,NaN
142,Czech Republic,124.0,70.0,2019-01-01,2019-01,NaN,NaN,NaN,NaN


In [227]:
df_inflation["Country"] = df_inflation["Country"].str.replace("Czechia", "Czech Republic")

In [231]:
df_adjusted["hotel_adj"] = df_adjusted["Hotel ceiling"] * (df_adjusted["inflation_index"] / 100)
df_adjusted["expenses_adj"] = df_adjusted["Daily allowance"] * (df_adjusted["inflation_index"] / 100)

In [233]:
df_adjusted

,Destination,Hotel ceiling,Daily allowance,Date,date_short,Country,TIME_PERIOD,inflation_index,Observation status (Flag),hotel_adj,expenses_adj
0,Belgium,148.00,102.00,2023-01-01,2023-01,Belgium,2023-01,164.87,NaN,244.007600,168.167400
1,Bulgaria,135.00,57.00,2023-01-01,2023-01,Bulgaria,2023-01,247.98,NaN,334.773000,141.348600
2,Czech Republic,124.00,70.00,2023-01-01,2023-01,Czech Republic,2023-01,208.50,NaN,258.540000,145.950000
3,Denmark,173.00,124.00,2023-01-01,2023-01,Denmark,2023-01,156.80,NaN,271.264000,194.432000
4,Germany,128.00,97.00,2023-01-01,2023-01,Germany,2023-01,163.00,NaN,208.640000,158.110000
...,...,...,...,...,...,...,...,...,...,...,...
617,Slovenia,110.00,60.00,2004-05-01,2004-05,Slovenia,2004-05,95.58,NaN,105.138000,57.348000
618,Slovakia,125.00,50.00,2004-05-01,2004-05,Slovakia,2004-05,94.12,NaN,117.650000,47.060000
619,Finland,140.98,92.34,2004-05-01,2004-05,Finland,2004-05,97.88,NaN,137.991224,90.382392
620,Sweden,141.77,92.91,2004-05-01,2004-05,Sweden,2004-05,97.35,NaN,138.013095,90.447885


In [240]:
df_adjusted.groupby("Country")[["Hotel ceiling", "Daily allowance", "hotel_adj","expenses_adj"]].mean().sort_values("expenses_adj", ascending=False)

,Hotel ceiling,Daily allowance,hotel_adj,expenses_adj
Country,,,,
Denmark,158.748261,117.873913,197.292585,146.481678
Finland,140.910435,106.000870,180.564904,136.880813
Sweden,168.187391,104.292609,214.315595,132.899135
United Kingdom,179.846842,105.087895,223.438994,130.754214
Austria,130.597391,95.061304,167.486992,123.193462
Netherlands,163.446957,94.990435,207.232979,121.167651
Belgium,140.140870,94.877391,178.159198,120.711777
Luxembourg,141.206957,93.043478,181.992226,119.679122
Ireland,152.128696,102.557391,176.396713,119.181752
